In [353]:
using LinearAlgebra
using Distributions
using FileIO
import JLD2
using DataFrames

In [354]:
using Revise
using Newtrinos
using Newtrinos.osc

In [355]:
osc_cfg = Newtrinos.osc.OscillationConfig(
    flavour=Newtrinos.osc.NNM(),
    propagation=Newtrinos.osc.Basic(),
    states=Newtrinos.osc.All(),
    interaction=Newtrinos.osc.SI()
    )

osc = Newtrinos.osc.configure(osc_cfg)

physics = (; osc);

In [356]:
experiments = (

   katrin= Newtrinos.katrin.configure(physics),
);

In [357]:
mass_nu=Newtrinos.osc.get_neutrinomass(osc_cfg.flavour)

(::Newtrinos.osc.var"#NeutrinoMassNND#125"{NNM}) (generic function with 1 method)

In [358]:
p = Newtrinos.get_params(experiments)

(N = 100.0, m₀ = 0.5, r = 1.0, Δm²₂₁ = 7.53e-5, Δm²₃₁ = 0.0024752999999999997, δCP = 1.0, θ₁₂ = 0.5872523687443223, θ₁₃ = 0.1454258194533693, θ₂₃ = 0.8556288707523761)

In [359]:
mass_nu_value= mass_nu(p)

0.3701187284175993

In [360]:
osc_cfg_SM = Newtrinos.osc.OscillationConfig(
    flavour=Newtrinos.osc.ThreeFlavour(),
    propagation=Newtrinos.osc.Basic(),
    states=Newtrinos.osc.All(),
    interaction=Newtrinos.osc.SI()
    )

osc = Newtrinos.osc.configure(osc_cfg_SM)

physics_SM = (; osc);

In [361]:
experiments_SM = (

   katrin= Newtrinos.katrin.configure(physics_SM),
);

In [362]:
NeutrinoMass_SM=Newtrinos.osc.get_neutrinomass_SM(osc_cfg_SM.flavour)

(::Newtrinos.osc.var"#NeutrinoMass_SM#124") (generic function with 1 method)

In [363]:
p = Newtrinos.get_params(experiments_SM)

(Δm²₂₁ = 7.53e-5, Δm²₃₁ = 0.0024752999999999997, δCP = 1.0, θ₁₂ = 0.5872523687443223, θ₁₃ = 0.1454258194533693, θ₂₃ = 0.8556288707523761)

In [364]:
mass_nu_value_SM= NeutrinoMass_SM(p)

0.010074612940900002

Calculation of m0 from m_nu measeurement in SM 

In [365]:
m_nu_squared=0.6 # eV^2, from KATRIN measurement

p= Newtrinos.get_params(experiments_SM)
U= Newtrinos.osc.get_PMNS(p)

sumU = 0.0

for i in 1:3
    sumU += abs(U[1,i])^2
end

term1= abs(U[1,2])^2*(p[:Δm²₂₁])
term2= abs(U[1,2])^2*(p[:Δm²₃₁])

m0_squared= (m_nu_squared-term1-term2) / sumU

m0=sqrt(m0_squared)

0.7741016790565695